# K-Nearest Neighbours

# K Nearest Neighbour Classification Algorithm

The K Nearest Neighbour (KNN) algorithm is a simple method for both classification and regression. With regards to classification, the algorithm assigns the class label to a sample based on the majority class of its nearest neighbors in the feature space.

### Euclidean Distance

In KNN, classification is done on the basis of nearest **Euclidean distance**. For two points $\mathbf{x} = (x_1, x_2, ..., x_n) $ and $ \mathbf{y} = (y_1, y_2, ..., y_n) $, the Euclidean distance is computed as:

$$
d(x, y) = \sqrt{\sum_{i=1}^{n} (x_i - y_i)^2}
$$

This formula calculates the straight-line distance between the points in $ n $-dimensional space.

### KNN Classification Algorithm

Given a training dataset, the KNN algorithm classifies a test datapoint based on the following steps:

1. For each test sample, calculate the Euclidean distance from the test datapoint to all training examples.
   
2. From these distances, choose $k$ training exmaples with minimum distances to new datapoint.

3. Classify the new point on the basis of the mode of the class labels of the $k$ training examples with min distance.

The classification rule for KNN can be written as:

$$
\hat{y}(\mathbf{x_i}) = \text{mode}_{1 \leq j \leq k} \left( \mathbf{y^j} \right)
$$

Where $y^j$ is a label of the $k$ nearest neighbors, and $\hat{y}(\mathbf{x_i})$ is the most frequent label within these k samples.

### Cross-Validation

In KNN, the choice of $k$ affects the model's performance. To determine the optimal $k$, **cross-validation** is used.

#### K-Fold Cross-Validation

K-fold cross-validation is a technique where the dataset is divided into $k$ equal folds. The model is trained on $k-1$ folds and tested on the remaining fold. This process is repeated for each fold, and the average performance (such as accuracy) is calculated.

The performance of the KNN classifier using cross-validation can be written as:

$$
\text{Accuracy} = \frac{1}{k} \sum_{i=1}^{k} \text{accuracy}(i)
$$

Where $\text{accuracy}(i)$ is the accuracy on the $i$-th fold, and $k$ is the total number of folds.


## Implementation

### Importing Libraries and Loading Data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import mode
%matplotlib inline

pd.set_option('future.no_silent_downcasting', True)

df = pd.read_csv('dataset.csv').iloc[:,1:]
df['class'].value_counts()

class
2    458
4    241
Name: count, dtype: int64

### Z-Score Normalization and Removing Outliers

In [2]:
df['bare_nuclei'] = pd.to_numeric(df['bare_nuclei'],errors="coerce")
df = df.dropna().astype('float64')

features = df.drop(columns=['class'])
features = (features-features.mean())/features.std()

outliers = (abs(features)>3).any(axis=1)
df = df[~outliers]
features = features[~outliers]
df.iloc[:,:-1] = features

In [3]:
df = df.reset_index()
df['class'] = df['class'].replace({2:0,4:1})
df

index  clump_thickness  unif_cell_size  unif_cell_shape  marg_adhesion  \
0        0         0.197760       -0.701698        -0.741230      -0.638897   
1        1         0.197760        0.277049         0.262591       0.757477   
2        2        -0.511269       -0.701698        -0.741230      -0.638897   
3        3         0.552274        1.582044         1.601018      -0.638897   
4        4        -0.156754       -0.701698        -0.741230       0.059290   
..     ...              ...             ...              ...            ...   
627    694        -0.511269       -0.701698        -0.741230      -0.638897   
628    695        -0.865783       -0.701698        -0.741230      -0.638897   
629    696         0.197760        2.234542         2.270232       0.059290   
630    697        -0.156754        1.582044         0.931805       0.408383   
631    698        -0.156754        1.582044         1.601018       0.757477   

     single_epith_cell  bare_nuclei  bland_chrom  norm_nucleoli   mitoses  \
0            -0.555202    -0.698341    -0.181694      -0.612478 -0.348145   
1             1.693925     1.771569    -0.181694      -0.284896 -0.348145   
2            -0.555202    -0.423907    -0.181694      -0.612478 -0.348145   
3            -0.105376     0.124962    -0.181694       1.353016 -0.348145   
4            -0.555202    -0.698341    -0.181694      -0.612478 -0.348145   
..                 ...          ...          ...            ...       ...   
627          -0.105376    -0.423907    -0.998122      -0.612478 -0.348145   
628          -0.555202    -0.698341    -0.998122      -0.612478 -0.348145   
629           1.693925    -0.149472     1.859375       2.335764  0.228998   
630          -0.105376     0.124962     2.675803       1.025434 -0.348145   
631           0.344449     0.399397     2.675803       0.370269 -0.348145   

     class  
0      0.0  
1      0.0  
2      0.0  
3      0.0  
4      0.0  
..     ...  
627    0.0  
628    0.0  
629    1.0  
630    1.0  
631    1.0  

[632 rows x 11 columns]

In [4]:
dat = df.to_numpy()
np.random.shuffle(dat)

### KNN Implementation

In [5]:
def KNN(X_train, X_test, y_train, k=1):
    distances = np.linalg.norm(X_train[:, np.newaxis] - X_test, axis=2)
    #print(distances.shape, "\n", distances, end="\n\n\n\n")
    
    nn_class = y_train[np.argsort(distances, axis=0)[:k]].astype(int)
    #print(nn_class.shape, "\n", nn_class, end="\n\n\n\n")
    
    predictions = mode(nn_class, axis=0).mode.flatten()
    
    return predictions
    

### Determining Optimal $k$ using K-Fold Cross-Validation 

In [6]:
folds = 10
max_acc = 0
max_k = 1
test = dat[550:,:]           # Testing dataset
dat = dat[:550,:]            # Training/Cross-Validation dataset
fold_size = dat.shape[0]//folds
for k in range(1,100,1):
    acc = 0
    for i in range(folds):
        X_test = dat[fold_size*i:fold_size*(i+1),:-1]
        y_test = dat[fold_size*i:fold_size*(i+1),-1]

        X_train = np.vstack((dat[:fold_size*i,:-1],dat[fold_size*(i+1):,:-1]))
        y_train = np.hstack((dat[:fold_size*i,-1],dat[fold_size*(i+1):,-1]))

        #print(X_test.shape, X_train.shape)
        #print(y_test.shape, y_train.shape)
    
        predictions = KNN(X_train, X_test, y_train, k=k)
        acc += (np.sum(predictions == y_test) / fold_size) * 100
    acc /= folds
    #print(acc)
    if acc > max_acc:
        max_acc = acc
        max_k = k

print("Maximum Accuracy:", max_acc, "for k =", max_k)

Maximum Accuracy: 88.18181818181816 for k = 1


### Final Testing

In [7]:
predictions = KNN(dat[:550,:-1], test[:,:-1], dat[:550,-1], k=max_k)
acc = (np.sum(predictions == test[:,-1]))/ test.shape[0] * 100
print("Final Accuracy:", acc)

Final Accuracy: 87.8048780487805
